<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter12/Detectron2_Chapter12_Tracing.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12 - Tracing

## Creating a model

In [ ]:
import torch
import torch.nn as nn

device = "cuda"
if not torch.cuda.is_available():
  device = "cpu"

class SimplePyTorchModel(nn.Module):
  def __init__(self):
    super(SimplePyTorchModel, self).__init__()
    self.linear = nn.Linear(4, 1)   
    self.linear.weight.data.fill_(0.01)
    self.linear.bias.data.fill_(0.01)

  def forward(self, x):
    return torch.relu(self.linear(x))

pt_model = SimplePyTorchModel()
pt_model.to(device)
print(pt_model)

SimplePyTorchModel(
  (linear): Linear(in_features=4, out_features=1, bias=True)
)


## Perform Predictions

In [ ]:
X = torch.tensor(
    data  = [[1, 2, 3, 4], 
           [-1, -2, -3, -4]], 
    dtype = torch.float32)
X = X.to(device)
with torch.no_grad():
  y = pt_model(X)
  print(y)

tensor([[0.1100],
        [0.0000]], device='cuda:0')


## Exporting to TorchScript

In [ ]:
dummy_X = torch.rand((1, 4))
dummy_X.to(device)
traced_model = torch.jit.trace(
    func            = pt_model.to("cpu"), 
    example_inputs  = dummy_X)
traced_model.to(device)

SimplePyTorchModel(
  original_name=SimplePyTorchModel
  (linear): Linear(original_name=Linear)
)

In [ ]:
for p in traced_model.parameters():
  print(p.data)

tensor([[0.0100, 0.0100, 0.0100, 0.0100]], device='cuda:0')
tensor([0.0100], device='cuda:0')


In [ ]:
print(traced_model.code)

def forward(self,
    x: Tensor) -> Tensor:
  linear = self.linear
  return torch.relu((linear).forward(x, ))



## Test the exported model

In [ ]:
with torch.no_grad():
  y = traced_model(X)
  print(y)

tensor([[0.1100],
        [0.0000]], device='cuda:0')


## Save the model

In [ ]:
traced_model.save("traced_model.pt")

## Test loading

In [ ]:
loaded_model = torch.jit.load("traced_model.pt")

In [ ]:
for p in loaded_model.parameters():
  print(p)

tensor([[0.0100, 0.0100, 0.0100, 0.0100]], device='cuda:0', requires_grad=True)
tensor([0.0100], device='cuda:0', requires_grad=True)


In [ ]:
loaded_model(X)

tensor([[0.1100],
        [0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)

## Download model

In [ ]:
from google.colab import files
files.download('traced_model.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>